<a href="https://colab.research.google.com/github/xborrat/NEFRoHack/blob/main/notebooks/mimic-iv_using_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MIMIC-IV

MIMIC-IV is a publicly accessible clinical database containing comprehensive information for hospital stays of over 60,000 patients. In particular, the database has a large focus on critical care, where patients are heavily monitored as they receive life saving interventions.

You can read more about MIMIC-IV from the following resources:

* [The MIMIC-IV PhysioNet project page](https://physionet.org/content/mimiciv/)
* [The MIMIC-IV online documentation](https://mimic-iv.mit.edu)

This notebook assumes that you have access to MIMIC-IV on Google BigQuery. If you're not sure if you do, or would like to apply for access, head over to the [MIMIC-IV access page](https://mimic-iv.mit.edu/docs/access/) for instructions.

## Load libraries and setup environment

In [ ]:
# Import libraries
from datetime import timedelta
import os

import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Make pandas dataframes prettier
from IPython.display import display, HTML, Image
%matplotlib inline

plt.style.use('ggplot')
plt.rcParams.update({'font.size': 20})

# Access data using Google BigQuery.
from google.colab import auth
from google.cloud import bigquery

We need to authenticate this notebook with Google Cloud Platform (GCP) in order to query MIMIC-IV. When prompted to login, **make sure you use the Google account granted access to MIMIC-IV via PhysioNet**. [Details on granting your Google account access are described in the online documentation](https://mimic-iv.mit.edu/docs/access/cloud/).

In [ ]:
# authenticate
auth.authenticate_user()

Next up, our final piece of configuration. In order to query the data, we need to set the project ID. This project ID is used to bill usage incurred by the querying. Don't worry: the queries we run are extremely lightweight, and usually cost pennies (if that).

If you're not sure what your project ID is, or if you haven't made a project on GCP, [you can read about creating and managing google projects here](https://cloud.google.com/resource-manager/docs/creating-managing-projects). Afterwards, change the `project_id` variable below to your Google project.

In [ ]:
# Set up environment variables
project_id = 'lcp-internal'
if project_id == 'CHANGE-ME':
  raise ValueError('You must change project_id to your GCP project.')
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id

# Read data from BigQuery into pandas dataframes.
def run_query(query, project_id=project_id):
  return pd.io.gbq.read_gbq(
      query,
      project_id=project_id,
      dialect='standard')

# set the dataset
# if you want to use the demo, change this to mimic_demo
dataset = 'mimic'

# test it works
df = run_query("""
SELECT subject_id
FROM `physionet-data.mimic_core.patients`
WHERE subject_id = 10012853
""")
assert df.shape[0] == 1, 'unable to query MIMIC-IV!'

If the above raises an error, you'll need to double check you've set your project correctly, and ensure that you have [requested access to MIMIC-IV on Google BigQuery via the PhysioNet project page](https://mimic-iv.mit.edu/docs/access/cloud/#accessing-mimic-iv-on-the-cloud).

We're going to go through the data in MIMIC-IV with an example patient. We'll select data for this patient using only their `subject_id`. After you've run through the notebook, you are welcome to change the `subject_id` to see the data for a different patient. This can be a useful exercise as not all patients have data recorded in every table! If this is your first time going through this notebook however, we recommend you keep the `subject_id` as is so the associated text blocks make sense.

In [ ]:
subject_id = 10012853

# Patients table

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_core.patients`
WHERE subject_id = {subject_id}
"""
pt = run_query(query)
pt['anchor_year'] = pt['anchor_year']
pt


Here we have information for an individual person. We have their genotypical sex in the `gender` column, which is 'F' for female. Their `anchor_age` is 91, and the corresponding `anchor_year` for this age is 2175. This `anchor_year` occurred sometime between 2014 - 2016. This is our temporal anchoring of the deidentified patient stay. We know that a measurement in the database which is recorded at the deidentified date of 2175-01-01 actually occurred sometime between 2014 and 2016.

This patient's particular age brings up an important deidentification step. HIPAA requires all ages over 89 to be obscured in order to protect patient identity, and leaves the method of obfuscation up to the individual data guardians. In MIMIC-IV, we have decided to set all ages over 89 to the value of 91, which is pretty close to the median age for that group. If you see a patient with an `anchor_age` of 91, then the only thing you know is that they were over 89 in the `anchor_year`. For this patient, we know they were older than 89 sometime between 2014 - 2016, which corresponds to 2175 in their deidentified time series. We've chosen 91 as it is a reasonable imputation for this age group. You can see the impact of this decision in a histogram of all ages.

In [ ]:
# Load admissions
query = f"""
SELECT anchor_age, count(*) as n
FROM `physionet-data.{dataset}_core.patients` pt
group by anchor_age
"""
age = run_query(query)
plt.figure(figsize=[12, 8])
plt.bar(age['anchor_age'], age['n'])
plt.xticks([x for x in range(0, 90, 10)] + [91])
plt.ylabel('Number of patients')
plt.xlabel('Age (years)')
plt.show()

Now you understand why there is such a huge jump from the number of 88 year olds to the number of "91" year olds! In fact, all patients >= 89 have their age set to 91.

N.B. If using the demo data, this jump isn't *that* obvious - but it is in the full dataset.

## Admissions

Next up, we can look at the hospitalizations for this patient.

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_core.admissions`
WHERE subject_id = {subject_id}
ORDER BY admittime
"""
adm = run_query(query)
adm

Here we see this patient has 6 admissions over a period of 4 years. Their initial admission occurs at their earliest `admittime`: 2175-04-05 15:36:00. Recall from the *patients* table that 2175 was their `anchor_year`; therefore we know that this admission occurred sometime between 2014 - 2016. Their final admission is in 2177, and by extension we know this admission must have occurred sometime between 2016 - 2018.

Going back to the first admission, we can see it lasted for about 5 days, and the patient was admitted to the hospital via the emergency room. We can get a sense of how often this happens by looking at adult patients who are admitted to the hospital.

In [ ]:
# Load admissions
query = f"""
SELECT admission_location, count(*) as n
FROM `physionet-data.{dataset}_core.admissions` a
WHERE admission_location IS NOT NULL
GROUP BY 1
ORDER BY 2
"""
df = run_query(query)
df['%'] = df['n'] / df['n'].sum() * 100.0

plt.figure(figsize=[8, 16])
plt.barh(df['admission_location'], df['n'])
plt.xlabel('Number of hospitalizations')
plt.xticks(rotation=45)
plt.show()

df

We can see around almost half of patients in MIMIC-IV are admitted via the emergency room. Let's go back to our patient.

In [ ]:
plt.figure(figsize=[12, 8])
for i, row in adm.iterrows():
  plt.plot([row['admittime'], row['dischtime']], [i, i], 'o-')
plt.ylabel('Admission number')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.show()
adm

The admission table also tells us about their demographics, has the time at which they were registered and discharged from the ED, and whether the patient died in the hospital. If we select their earliest hospitalization, we can use the *transfers* table to track their in-hospital movement, i.e. which wards they stayed in during their hospital stay.

In [ ]:
# Load admissions
query = f"""
SELECT *
FROM `physionet-data.{dataset}_core.transfers` tr
WHERE hadm_id IN
(
  SELECT hadm_id
  FROM `physionet-data.{dataset}_core.admissions`
  WHERE subject_id = {subject_id}
  ORDER BY admittime
  LIMIT 1
)
ORDER BY intime
"""
tr = run_query(query)
tr

The patient was admitted to the emergency department (ED) and spent just under 10 hours in the ED. She then spent five days in a medicine ward until being discharged from the hospital. From the admissions table above, we know that she was discharged home from the hospital (`discharge_location` of 'HOME').

As a large amount of data in MIMIC-IV is focused on a patient's ICU stay, it behooves us to select a hospitalization which involved an ICU stay. To figure out what stays are ICU stays, let's group *transfers* by the careunit.

In [ ]:
# Load admissions
query = f"""
SELECT careunit, count(*) AS n
FROM `physionet-data.{dataset}_core.transfers` tr
WHERE careunit IS NOT NULL
GROUP BY 1
ORDER BY 2
"""
df = run_query(query)

plt.figure(figsize=[8, 16])
plt.barh(df['careunit'], df['n'])
plt.xlabel('Number of stays')
plt.xticks(rotation=45)
plt.show()

Above we can see that by far the majority of stays are in the emergency room. This is because the emergency room in the hospital is better thought of as a clinic: many patients have many brief stays.

So, now that we know all the care units, we can simply subselect our patient's hospitalizations to those which contain an ICU stay. Note that patients in MIMIC-IV are *not* guaranteed to have an ICU stay, but we've picked this patient as we know they do have one.

In [ ]:
# Load admissions
query = f"""
SELECT *
FROM `physionet-data.{dataset}_core.transfers` tr
WHERE hadm_id IN
(
  SELECT hadm_id
  FROM `physionet-data.{dataset}_core.transfers`
  WHERE subject_id = {subject_id}
  AND careunit IN
  (
    'Neuro Surgical Intensive Care Unit (Neuro SICU)',
    'Coronary Care Unit (CCU)',
    'Trauma SICU (TSICU)',
    'Surgical Intensive Care Unit (SICU)',
    'Cardiac Vascular Intensive Care Unit (CVICU)',
    'Medical/Surgical Intensive Care Unit (MICU/SICU)',
    'Medical Intensive Care Unit (MICU)'
    -- exclude NICU - adult patients do not have stays in the NICU
    -- 'Neonatal Intensive Care Unit (NICU)'
  )
)
ORDER BY intime
"""
tr = run_query(query)

# retain hadm_id for later use in queries
hadm_id = tr['hadm_id'].values[0]

# plot the patient stays
plt.figure(figsize=[12, 8])
for i, row in tr.iterrows():
  if pd.isnull(row['outtime']):
    continue
  plt.plot([row['intime'], row['outtime']], [i, i], 'o-', label=row['careunit'])
plt.ylabel('Ward stay')
plt.xlabel('Date')
plt.xticks(rotation=45)
plt.legend(loc='lower right')
plt.show()

tr

As it turns out, this patient has only one hospitalization which contains an ICU stay. The *transfers* table provides us with the patient's full hospital course: they were admitted via the emergency department to a medicine ward in the hospital. As the deidentification approach retains the time of day, we know the patient was transfered at 2:34 AM from the medicine ward to the trauma/surgical intensive care unit (TSICU). The patient spent approximately three days in the TSICU. Note that there are two rows associated with the same care unit (TSICU). This is not an error, and this phenomenon can occur when a patient moves from one physical location to another which have the same designation. In this case, it is likely that the patient moved beds within the same physical location, though it is also possible that the TSICU care unit physically corresponds to more than one room, and the patient was moved between the two rooms. After their stay in the ICU, the patient was transferred to a vascular ward in the hospital. After four days in the vascular ward, the patient is discharged from the hospital. As before, we can investigate their hospital level information in the admissions table.

In [ ]:
# Load admissions
query = f"""
SELECT *
FROM `physionet-data.{dataset}_core.admissions`
WHERE hadm_id = {hadm_id}
"""
adm = run_query(query)
adm.T

Above, we can see that the patient was an emergency admission admitted via the emergency room (or emergency department, the terms are usable interchangeably). In this case, the patient is discharged to a skilled nursing facility.

Hopefully you now understand how the *core* dataset can be used to identify and understand patient movement within the hospital. We'll retain this patient's identifiers (`subject_id` and `hadm_id`), and move on to highlighting the information available for the patient in the *hosp*, *icu*, and *cxr* datasets.

## *ed*

Since the patient was admitted through the emergency department, we very likely have their data from the ED! (Very likely because MIMIC-ED and MIMIC-IV do not overlap completely).

Let's check!

In [ ]:
query = f"""
SELECT * FROM `physionet-data.{dataset}_ed.edstays`
where subject_id = {subject_id}
ORDER BY intime
"""
edstays = run_query(query)
edstays

We do indeed have quite a few ED stays for this patient. We can subselect to the ED stay associated with the hospitalization we chose earlier by filtering on the `hadm_id`.

In [ ]:
query = f"""
SELECT * FROM `physionet-data.{dataset}_ed.edstays`
where hadm_id = {hadm_id}
ORDER BY intime
"""
edstays = run_query(query)

ed_stay_id = edstays['stay_id'].values[0]

edstays

In [ ]:
query = f"""
SELECT * FROM `physionet-data.{dataset}_ed.triage`
where stay_id = {ed_stay_id}
"""
triage = run_query(query)
display(triage)

Above we can see that this patient had a chief complaint of a deep vein thrombosis (DVT) and was a transfer patient via the ED. Her vital signs are also shown; note the high systolic blood pressure!

In [ ]:
query = f"""
SELECT * FROM `physionet-data.{dataset}_ed.vitalsign`
where stay_id = {ed_stay_id}
order by charttime
"""
vs = run_query(query)
display(vs)

The *vitalsign* table contains vital signs noted by nursing staff while the patient was in the ED. Usually you'll see a set of vital signs every 1-4 hours, but in this case it seems the patient was being monitored a bit more frequently; possible because she appeared unwell. Note the reasonably low oxygen saturation (`o2sat`) of 93%. You'll also see most of the columns in this table mirror those of triage.

In [ ]:
query = f"""
SELECT * FROM `physionet-data.{dataset}_ed.medrecon`
where stay_id = {ed_stay_id}
"""
medrecon = run_query(query)
display(medrecon)

Above we can see the medicine reconciliation table, or *medrecon*. This is a list of medications which the patient is taking at home. In this case, the patient is on 11 medications, ranging from aspirin to methimazole. Note also that the `etccode` and `etcdescription` columns provide a useful classification of the type of medication. There may be more than one `etccode` for a single medication, and the `etc_rn` column helps us differentiate between those medications.

In [ ]:
query = f"""
SELECT * FROM `physionet-data.{dataset}_ed.diagnosis`
where stay_id = {ed_stay_id}
"""
eddx = run_query(query)
display(eddx)

Finally the *diagnosis* table provides us with the diagnoses which the patient was billed for **after leaving the ED**.

## *hosp*

The *hosp* dataset contains information sourced from the hospital electronic health record (EHR) system. There are many tables in this dataset, but we can group the data contained as follows:

* Service data: the hospital service the patient was under during their hospital stay, e.g. medicine, surgery, etc.
  * *services*
* Order data: orders made for the patient including treatments, code status, and so on
  * *poe*, *poe_detail*
* Microbiology studies: blood cultures, antibiotic sensitivities, and other similar tests
  * *d_micro*, *microbiologyevents*
* Laboratory data: information measured on a specimen derived from the patient, e.g. the electrolytes in a sample of patient blood
  * *d_labitems*, *labevents*
* Medication data: information about the prescription and administration of drugs/compounds to the patient
  * *prescriptions*, *pharmacy*, *emar*, *emar_detail*
* Billing data: trained personelle assign coded terms to a patient's hospitalization to facilitate the billing process
  * *d_hcpcs*, *hcpcsevents*, *d_icd_diagnoses*, *diagnoses_icd*, *d_icd_procedures*, *procedures_icd*, *drgcodes*

We'll now go through these tables and subselect to our example patient.

### *services*

In [ ]:
# services
query = f"""
WITH t1 AS
(
SELECT hadm_id, count(*) as n
FROM `physionet-data.{dataset}_hosp.services`
GROUP BY 1
)
SELECT n AS n_service
, count(*) AS n
FROM t1
GROUP BY 1
ORDER BY 1
"""
df = run_query(query)
df['%'] = df['n'] / df['n'].sum() * 100.0
display(df.head())

query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.services`
WHERE hadm_id = {hadm_id}
"""
serv = run_query(query)
serv.T

The *services* table is fairly straightforward. Around 94% of patients are assigned a single service for their entire hospital stay. In this case, the patient has the "MED" service, which is short for medical.

In [ ]:
query = f"""
SELECT curr_service AS service, count(*) as n
FROM `physionet-data.{dataset}_hosp.services`
GROUP BY 1
ORDER BY 2
"""
df = run_query(query)

plt.figure(figsize=[8, 16])
plt.barh(df['service'], df['n'])
plt.xlabel('Number of stays')
plt.xticks(rotation=45)
plt.show()

### *poe*, *poe_detail*

There is a soft convention in MIMIC-IV tables that a "_detail" table provides more information for an identifier in its corresponding primary table. In this case, the *poe* table provides information about provider orders, and the *poe_detail* table provides further information about the orders in the *poe* table. An example will hopefully elucidate this further.

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.poe`
WHERE hadm_id = {hadm_id}
ORDER BY poe_seq
"""
poe = run_query(query)
poe[['subject_id', 'poe_id', 'poe_seq', 'ordertime', 'order_type', 'order_subtype']]

The patient has 198 orders over their stay. The `order_type` and `order_subtype` give us an idea of what type of orders are made.

In [ ]:
poe.groupby(['order_type', 'order_subtype'])[['poe_id']].count()

We can see:

1. There are a number of orders for the management of the patients movement in the hospital (commonly abbreviated ADT for admission/discharge/transfer)
2. The patient has had physical therapy consults
3. Various imaging studies have been ordered including CT scans and x-rays
4. The patient was on non-invasive ventilation (BiPAP order)


In [ ]:
query = f"""
SELECT p.*, pd.field_name, pd.field_value
FROM `physionet-data.{dataset}_hosp.poe` p
INNER JOIN `physionet-data.{dataset}_hosp.poe_detail` pd
  ON p.poe_id = pd.poe_id
WHERE hadm_id = {hadm_id}
ORDER BY poe_seq
"""
poedet = run_query(query)
poedet[['subject_id', 'poe_id', 'ordertime', 'order_type', 'order_subtype', 'field_name', 'field_value']]

### *d_micro*, *microbiologyevents*

Another commonly found convention in MIMIC-IV is the use of an "events" table with a "d_" table. This is a type of database normalization, where information is encoded as an integer and stored elsewhere.

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.microbiologyevents`
WHERE hadm_id = {hadm_id}
"""
micro = run_query(query)
micro

As it turns out, this patient did not have any microbiology information during this hospitalization. This is to be expected - not all patients receive every test! We can look across the patient's entire record, rather than just within this hospitalization.

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.microbiologyevents`
WHERE subject_id = {subject_id}
ORDER BY chartdate, charttime
"""
micro = run_query(query)
micro

The patient has 65 records in *microbiologyevents* table. Recall that the patient's admission for the hospital stay we are examining occurred on 2176-11-25 21:28:00. We can see that about two months prior to the admission (2176-10-10), the patient had a urine culture (`test_name`), which tested positive for multiple organisms (`org_itemid` is non-null). This raises an important point about the microbiology table and data in the *hosp* dataset in general: while the information is sourced from the hospital, **the dataset also contains data collected outside of the hospital**. This is particularly useful when you'd like to search for baseline data prior to their hospitalization.

To better understand the micro data, let's subselect to urine culture specimen.

In [ ]:
micro.loc[micro['micro_specimen_id'] == 8328091].T

Above we can see we actually have many rows for a single `micro_specimen_id`. This is expected, and there are a few different ways which data can present itself in this table. Recall that the intention of a microbiology test is to determine if an organism is present in a sample. In this case, the same is urine. The possibilities are:

* If no growth is found, the `org_itemid` and subsequent columns will be NULL, and there will only be a single row for the `micro_specimen_id`
* If an organism is found, then each organism will be present as a single row. Each unique organism is assigned a unique `org_itemid`, and *d_micro* provides the interpretable name for these organisms.
* If antibiotics are tested on a given organism, then additional rows will be present for combination of antibiotic and organism (i.e. a single antibiotic denoted by `ab_itemid` may be tested on the one or more organisms which grew). Antibiotic parameters and sensitivities are present in the remaining columns (`dilution_text`, `dilution_comparison`, `dilution_value`, `interpretation`).

Back to our patient, we can re-run a query joining to *d_micro* to get the interpretable names of the organisms and antibiotics associated with the specimen.

In [ ]:
micro_specimen_id = 8328091

query = f"""
SELECT
me.subject_id, me.hadm_id
, me.micro_specimen_id
, me.charttime, me.storetime
, me.test_seq
, me.test_name
, me.test_itemid
, test.label as test_label
, me.org_itemid
, org.label as org_label
, me.isolate_num, me.quantity
, me.ab_itemid
, ab.label as ab_label
, me.dilution_text, me.dilution_comparison
, me.dilution_value, me.interpretation
FROM `physionet-data.{dataset}_hosp.microbiologyevents` me
INNER JOIN `physionet-data.{dataset}_hosp.d_micro` test
  ON me.test_itemid = test.itemid
INNER JOIN `physionet-data.{dataset}_hosp.d_micro` org
  ON me.org_itemid = org.itemid
INNER JOIN `physionet-data.{dataset}_hosp.d_micro` ab
  ON me.ab_itemid = ab.itemid
WHERE micro_specimen_id = {micro_specimen_id}
ORDER BY charttime, org_itemid, ab_itemid
"""
micro = run_query(query)

# only display a subset of columns for conciseness
print('Columns in table: {}'.format(', '.join(micro.columns)))
cols = ['subject_id', 'charttime', 'storetime', 'test_name', 'org_label', 'ab_label',
        'dilution_text', 'interpretation']
micro[cols]

Above, we can see there are two organisms found in the urine culture: morganella morganii and providencia stuartii. Both organisms have been tested for sensitivity against a number of antibiotics (nitrofurantoin, gentamicin, tobramycin, ciprofloxacin). Both organisms are resistant ('R') to nitrofurantoin (note the distinct dilutions used!), but susceptible to all other antibiotics ('S').

### *d_labitems*, *labevents*

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.labevents`
WHERE hadm_id = {hadm_id}
ORDER BY charttime
"""
lab = run_query(query)
lab

Laboratory data contains measurements made from human derived specimens (urine samples, blood samples, sputum samples, etc). When a sample is drawn, it is assigned a unique `specimen_id`. Measurements made from this sample will be associated with the same `specimen_id`. For example, a measurement of sodium and potassium made on the same blood sample will appear as two rows with the same `specimen_id`.

In order to understand the measurements made, it's useful to join the *labevents* table to the *d_labitems* table, which has descriptive information for each type of lab measurement.

In [ ]:
query = f"""
SELECT le.hadm_id
, le.specimen_id
, le.charttime
, le.storetime
, le.itemid
, dl.fluid
, dl.category
, dl.label
, le.value
, le.valuenum
, le.valueuom
, le.ref_range_lower, le.ref_range_upper
, le.flag, le.priority
, le.comments
FROM `physionet-data.{dataset}_hosp.labevents` le
INNER JOIN `physionet-data.{dataset}_hosp.d_labitems` dl
  ON le.itemid = dl.itemid
WHERE hadm_id = {hadm_id}
ORDER BY charttime, label
"""
lab = run_query(query)
lab

We can now see the added information from *d_labitems*: each lab measurement (`itemid`) is associated with a fluid (the type of specimen it corresponds to) and a category (broad description of the type of lab). It's important to factor in this information: though two labs may be labelled as "pH", one may refer to urine pH while the other refers to blood pH. The value of these measurements have very different reference ranges and interpretations - and this is one of the reasons why MIMIC-IV stores measurements with an `itemid` rather than a textual label.

In [ ]:
plt.figure(figsize=[16, 8])

# labs
lab_labels = [
  'White Blood Cells',
  'Hemoglobin',
  'Potassium',
  'Creatinine',
  'Magnesium'
]
for label in lab_labels:
  idx = (lab['label'] == label) & (lab['fluid'] == 'Blood')
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'valuenum'], 'o--',  label=f'{label}')

plt.legend(fontsize=16, loc='lower right')
#plt.xlim(['2110-09-07 00:00:00', '2110-09-09 00:00:00'])
plt.xticks(rotation=45)

# shade in their ICU stay
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [10, 10], color='#ababab', alpha=0.5)

plt.show()

Everything looks pretty normal here! Let's take a look at a different type of lab: a blood gas. These are bedside measurements of blood which provide primarily inform about the partial pressure of various gases in the blood (e.g. oxygen, carbon dioxide).

In [ ]:
lab.loc[lab['category'] == 'Blood Gas', 'label'].value_counts()

In [ ]:
plt.figure(figsize=[16, 8])

# labs
lab_labels = [
  'Base Excess',
  'pO2',
  'pH',
  'pCO2',
  'Calculated Total CO2',
  'Lactate',
  'Free Calcium',
]

for label in lab_labels:
  idx = (lab['label'] == label) & (lab['fluid'] == 'Blood')
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'valuenum'], 'o--',  label=f'{label}')

# add specimen type
for i, row in lab.loc[lab['label'] == 'Specimen Type'].iterrows():
  plt.text(row['charttime'], 20, row['value'], rotation=60, ha='center')

plt.legend(fontsize=16, loc='lower right')
#plt.xlim(['2110-09-07 00:00:00', '2110-09-09 00:00:00'])
plt.xticks(rotation=45)

# shade in their ICU stay
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [100, 100], color='#ababab', alpha=0.5)

plt.show()

# same plot, but ignore a few labels
plt.figure(figsize=[16, 8])

# labs
lab_labels = [
  'Base Excess',
  'pH',
  'Lactate',
  'Free Calcium',
]

for label in lab_labels:
  idx = (lab['label'] == label) & (lab['fluid'] == 'Blood')
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'valuenum'], 'o--',  label=f'{label}')

# add specimen type
for i, row in lab.loc[lab['label'] == 'Specimen Type'].iterrows():
  plt.text(row['charttime'], 10, row['value'], rotation=60, ha='center')

plt.legend(fontsize=16, loc='lower right')
#plt.xlim(['2110-09-07 00:00:00', '2110-09-09 00:00:00'])
plt.xticks(rotation=45)

# shade in their ICU stay
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [15, 15], color='#ababab', alpha=0.5)

plt.show()

### *prescriptions*, *pharmacy*, *emar*, *emar_detail*

These tables record medications given to the patient.

* The *prescriptions* table tracks medication orders, i.e. the intention for a patient to receive a medication (and almost all do receive it!)
* The *pharmacy* table provides further detail about the formulation of the prescribed medication
* The *emar* and *emar_detail* tables, short of electronic medication administration record, record actual administration of medications

The *emar* and *emar_detail* are sourced from barcode scanning of physical medications at the time of their administration. As a result, they provide a rich and highly granular record of a patient's medications. Note however that eMAR data is not available for all patients as the system was implemented during the collection period of MIMIC-IV. The patient we have selected does have eMAR during their hospital stay, which we can confirm with a query.

First, let's look at snapshots of these tables.

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.prescriptions`
WHERE hadm_id = {hadm_id}
ORDER BY starttime
"""
pr = run_query(query)
display(pr.head())

query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.pharmacy`
WHERE hadm_id = {hadm_id}
ORDER BY starttime
"""
ph = run_query(query)
display(ph.head())

query = f"""
SELECT *
FROM `physionet-data.{dataset}_hosp.emar`
WHERE hadm_id = {hadm_id}
ORDER BY charttime
"""
emar = run_query(query)
display(emar.head())

query = f"""
SELECT det.*
FROM `physionet-data.{dataset}_hosp.emar_detail` det
INNER JOIN `physionet-data.{dataset}_hosp.emar` e
  ON det.emar_id = e.emar_id
WHERE e.hadm_id = {hadm_id}
ORDER BY emar_seq
"""
emardet = run_query(query)
display(emardet.head())

As the patient is prescribed and administered heparin and enoxaparin (both anticoagulants), they serve as a useful illustration of the differences between these tables. First, we can look at the prescriptions for these medications.

In [ ]:
meds = ['Heparin', 'Heparin Sodium', 'Enoxaparin Sodium']
colors = {
    'Heparin': '#1b9e77',
    'Heparin Sodium': '#1b9e77',
    'Enoxaparin Sodium': '#d95f02'
}
idx = pr['drug'].isin(meds)
df = pr.loc[idx].copy()

plt.figure(figsize=[12, 8])
for med in meds:
  color = colors[med]
  idx = df['drug'] == med
  n = 0
  for i, row in df.loc[idx].iterrows():
    if n > 0:
      # do not add to the legend if we have already plotted at least once
      med = '__no_legend__'

    xi = [row['starttime'], row['stoptime']]
    dose = row['dose_val_rx']
    if '-' in dose:
      # plot upper/lower ranges
      dose_lower, dose_upper = dose.split('-')
      dose_lower = int(dose_lower)
      dose_upper = int(dose_upper)
      plt.fill_between(xi, [dose_lower, dose_lower], [dose_upper, dose_upper], color=color, alpha=0.5)
    else:
      dose = pd.to_numeric(dose.replace(',', ''))
      plt.plot(xi, [dose, dose], 'o-', color=color, label=med)
    n += 1


# shade in their ICU stay
upper = 25500
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [upper, upper], color='#ababab', alpha=0.5)


plt.legend()
plt.xticks(rotation=45)
plt.show()

In [ ]:
idx = emar['medication'].isin(['Heparin', 'Enoxaparin Sodium'])
df_emar = emar.loc[idx].copy()

# dose due is in the "parent" of the emar detail group
# where parent_field_ordinal is null
# only 1 of these per emar_id
idx = emardet['parent_field_ordinal'].isnull()
cols = [
  'emar_id', 'administration_type', 'dose_due', 'dose_due_unit',
  'complete_dose_not_given', 'will_remainder_of_dose_be_given'
]
df_emar = df_emar.merge(emardet.loc[idx, cols], on='emar_id', how='left')
df_emar['dose_due'] = pd.to_numeric(df_emar['dose_due'])

# add in the actual dose administered
# since it can be across multiple formulary doses (e.g. multiple pills)
# we aggregate the detail table
# in heparin/enoxaparin case, we are probably aggregating over a single adm
# note that for heparin, since it's a continuous infusion, dose_given is 0/None
idx = emardet['parent_field_ordinal'].notnull()
cols = ['emar_id', 'dose_given', 'dose_given_unit', 'product_amount_given', 'product_unit']
em = emardet.loc[idx, cols].copy()

# re-cast columns - sometimes drugs have string doses
em['dose_given'] = pd.to_numeric(em['dose_given'], errors='coerce')
em['product_amount_given'] = pd.to_numeric(em['product_amount_given'], errors='coerce')

em = em.groupby('emar_id').agg(
  dose_given=pd.NamedAgg(column='dose_given', aggfunc='sum'),
  dose_given_unit=pd.NamedAgg(column='dose_given_unit', aggfunc='max'),
  product_amount_given=pd.NamedAgg(column='product_amount_given', aggfunc='sum'),
  product_unit=pd.NamedAgg(column='product_unit', aggfunc='max'),
)

df_emar = df_emar.merge(em, on='emar_id', how='left')
df_emar.sort_values(['charttime', 'emar_seq'], inplace=True)

# display a subset of the columns
cols = ['hadm_id', 'emar_id', 'pharmacy_id', 'charttime', 'medication',
        'administration_type', 'scheduletime', 'storetime',
        'dose_due', 'dose_due_unit', 'dose_given', 'dose_given_unit']
df_emar[cols].head(n=5)

Above, we've combined detailed information about the eMAR administrations (primarily to get the dose given) with the main information about the administrations in the *emar* table.

We can see a lot of information here: when heparin was started, stopped, when dosing was confirmed, and so on. We also can see the `dose_due`. In this case, `dose_given` is 0, and this appears to happen as this is a continuous infusion rather than an individual dose.

Note that the `pharmacy_id` gives us a link to the associated record in the *pharmacy* table, which we can look up pretty easily:

In [ ]:
ph.loc[ph['pharmacy_id'].isin([76768838, 79252552]), ['hadm_id', 'pharmacy_id', 'starttime', 'stoptime', 'medication', 'proc_type', 'route', 'frequency']]

We can similarly find the orders in prescriptions. In the below example, we're highlighting how you can find drugs using the name of the drug, though note that a few of the `pharmacy_id` again match the administrations above.

In [ ]:
idx = pr['drug'].isin(['Heparin', 'Heparin Sodium'])
pr.loc[idx, ['subject_id', 'pharmacy_id', 'starttime', 'stoptime', 'drug', 'dose_val_rx']]

In [ ]:
plt.figure(figsize=[12, 8])
for med in df_emar['medication'].unique():
  idx = df_emar['medication'] == med
  plt.plot(df_emar.loc[idx, 'charttime'], df_emar.loc[idx, 'dose_due'], 'o', label=med)


# shade in their ICU stay
upper = 1200
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [upper, upper], color='#ababab', alpha=0.5)


plt.legend()
plt.xticks(rotation=45)
plt.show()

Finally, we can combine the medication information with other information (e.g. labs). Often anticoagulants are titrated based on a coagulation factor in the blood, so let's plot these medications with those factors.

In [ ]:
plt.figure(figsize=[12, 8])
meds = {
    'Heparin': 'Heparin (hecto-units/hour)',
    'Enoxaparin Sodium': 'Enoxaparin (centigrams)',
}
for med, label in meds.items():
  idx = df_emar['medication'] == med
  if med == 'Heparin':
    multiplier = 0.01
  else:
    multiplier = 0.1
  plt.plot(df_emar.loc[idx, 'charttime'], df_emar.loc[idx, 'dose_due'] * multiplier, 'o', label=label)


# plot the lab factors
for label in ['PT', 'INR(PT)']:
  idx = lab['label'] == label
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'valuenum'], 'o-', label=label)

# shade in their ICU stay
upper = 30
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [upper, upper], color='#ababab', alpha=0.5)

plt.legend()
plt.xticks(rotation=45)
plt.show()

**Note that heparin/enoxaparin administrations have special units**. Heparin is usually in units/hour, we've plotted hecto-units/hour (1/100 units). Similarly we've plotted centigrams rather than milligrams (1/10 mg).

Without knowing the reference ranges of the coagulation measures, it's hard to know whether this is OK! We can add those to the plot using the reference ranges from the lab table.



In [ ]:
plt.figure(figsize=[12, 8])
meds = {
    'Heparin': 'Heparin (hecto-units/hour)',
    'Enoxaparin Sodium': 'Enoxaparin (centigrams)',
}
for med, label in meds.items():
  idx = df_emar['medication'] == med
  if med == 'Heparin':
    multiplier = 0.01
  else:
    multiplier = 0.1
  plt.plot(df_emar.loc[idx, 'charttime'], df_emar.loc[idx, 'dose_due'] * multiplier, 'o', label=label)


# plot the lab factors
for label in ['PT', 'INR(PT)']:
  idx = lab['label'] == label
  # plot the lab itself
  p = plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'valuenum'], 'o-', label=label)

  # plot the lower/upper reference range
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'ref_range_lower'], 'x--', color=p[0].get_color(), label='__no_legend__')
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'ref_range_upper'], 'x--', color=p[0].get_color(), label='__no_legend__')

# print the reference ranges
idx = lab['label'].isin(['PT', 'INR(PT)'])
display(lab.loc[idx, ['label', 'ref_range_lower', 'ref_range_upper']].drop_duplicates())

# shade in their ICU stay
upper = 30
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [upper, upper], color='#ababab', alpha=0.5)

plt.legend()
plt.xticks(rotation=45)
plt.show()

From this it's much clearer that this patient has very high levels of anticoagulation, and that her blood takes significantly longer to plot than the reference ranges. This may be due to a disease process or a therapeutic treatment for certain conditions (atrial fibrillation, etc). In this case, as the patient came in with a DVT, we can be fairly sure this is therapeutic anticoagulation.


### *d_hcpcs*, *hcpcsevents*, *d_icd_diagnoses*, *diagnoses_icd*, *d_icd_procedures*, *procedures_icd*, *drgcodes*

The billing code tables record information about diagnoses and procedures for which a patient was billed. They are often used as surrogates for the diseases or conditions a patient has, though caution is warranted as billed items do not perfectly account for all procedures and diseases present. For example, obesity is common in the ICU cohort as measured patient body mass index (over 30% of patients are obese), but rare if checked with the diagnostic billing code for obesity (~10%). We can look at the diagnosis table for this hospitalization to get an idea of how the billing tables work.

In [ ]:
query = f"""
SELECT dx.*, di.long_title
FROM `physionet-data.{dataset}_hosp.diagnoses_icd` dx
INNER JOIN `physionet-data.{dataset}_hosp.d_icd_diagnoses` di
  ON dx.icd_code = di.icd_code
WHERE hadm_id = {hadm_id}
ORDER BY dx.seq_num
"""
dx = run_query(query)
dx

Above we can see that there are many diagnoses for this visit (29!). The `seq_num` column provides an ordering for diagnoses. The order does have some meaning; the first diagnosis is usually considered "primary", but disease specific coding practices may impact this. In this case, we can see the patient was admitted for an acute thrombosis (`seq_num` = 1) with associated respiratory failure (`seq_num` = 2). Other conditions were likely chronic and occuring prior to their hospitalization (e.g. obstructive sleep apnea), while some diseases are certainly acute and localized to the hospital stay (i.e. the embolism).

In [ ]:
query = f"""
SELECT drg.*
FROM `physionet-data.{dataset}_hosp.drgcodes` drg
WHERE hadm_id = {hadm_id}
"""
drg = run_query(query)
drg

In [ ]:
query = f"""
SELECT dx.*, di.long_title
FROM `physionet-data.{dataset}_hosp.procedures_icd` dx
INNER JOIN `physionet-data.{dataset}_hosp.d_icd_procedures` di
  ON dx.icd_code = di.icd_code
WHERE hadm_id = {hadm_id}
ORDER BY dx.seq_num
"""
proc = run_query(query)
print(proc.iloc[0, -1])
proc

Surprisingly there is only one procedure with the patient hospitalization - a ventilation event which lasted over 96 hours.

In [ ]:
query = f"""
SELECT h.*
FROM `physionet-data.{dataset}_hosp.hcpcsevents` h
WHERE hadm_id = {hadm_id}
"""
hcpcs = run_query(query)
hcpcs

And finally, there are no HCPCS associated data with this patient.

### *icu*

The *icu* dataset contains information primarily sourced from the MetaVision clinical information system (iMDSoft). MetaVision is used to document most measurements, treatments, and ongoing processes for a patient while they stay in the ICU. The *icu* dataset has an *icustays* table which defines individual patient stays in the ICU (based upon *transfers*), and the remaining events tables which contain patient data. The tables are in a "star" schema, where the *d_items* table defines concepts indexed by the `itemid` column, and the remaining tables contain patient data for each `itemid`. The tables in the *icu* dataset are:

* *icustays*
* *d_items*
* Events tables:
  * *chartevents*
  * *datetimeevents*
  * *inputevents*
  * *outputevents*
  * *procedureevents*

### *icustays*

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_icu.icustays`
WHERE hadm_id = {hadm_id}
"""
ie = run_query(query)

# retain stay_id for later queries
stay_id = ie['stay_id'].values[0]

ie

For this patient's hospitalization, we have a single ICU stay of almost 4 days. The patient was admitted to the ICU at `intime` (2176-11-26 02:34:49) and was discharged from the ICU at `outtime` (2176-11-29 20:58:54). Note that unlike the *transfers* table, the *icustays* table has only one row for the patient's ICU stay. This reflects an underlying assumption of the table where consecutive physical stays in the ICU are merged into a single encounter.

ICU encounters are indexed by the `stay_id` column. This uniquely identifies this patient's ICU stay.

### *chartevents* and *d_items*

Rather than individual investigate each event table, it's easiest to understand *chartevents*, and subsequently contrast the other tables to it. *chartevents* is the largest of the data tables, and contains most information documented during a patient's stay.

In [ ]:
# Load admissions
query = f"""
SELECT *
FROM `physionet-data.{dataset}_icu.chartevents`
WHERE stay_id = {stay_id}
ORDER BY charttime, itemid
"""
ce = run_query(query)
print(ce.shape)
ce.head()

During the patient's four days in the ICU over 4,000 observations were documented, highlighting how heavily monitored patients are. The `subject_id`, `hadm_id`, and `stay_id` columns are patient, hospital, and ICU stay identifiers as we are famiiar with. `charttime` contains the time at which a measurement corresponds to, whereas `storetime` contains when the measurement was stored in the system. The `itemid` column stores an integer which uniquely identifies concepts such as heart rate, oxygen saturation, and so on. The `value` column stores the patient measurement made, and if it is numeric, the `valuenum` column will contain it cast to a numeric data type. The `valueuom` contains the unit of measurement associated with the value (if appropriate). Finally, the `warning` column indicates if staff added a warning to the value.

To get an idea of what type of data is present in chartevents, we can join this table to *d_items* to get the `label` for each itemid, and do a simple count of the number of rows.

In [ ]:
query = f"""
SELECT *
FROM `physionet-data.{dataset}_icu.d_items`
ORDER BY itemid
"""
di = run_query(query)

print('Our top 5 rows and their associated definitions.')

display(ce[['stay_id', 'charttime', 'itemid', 'value', 'valueuom']].head(n=4))

itemid = ce['itemid'].head(n=4).values
display(di.loc[di['itemid'].isin(itemid), ['itemid', 'label', 'category', 'linksto']].sort_values('itemid'))

ce.merge(di, on='itemid', how='inner')['label'].value_counts()[:30]

We can see the most frequently documented item are the safety measures in place around the patient, followed by vital signs, measurements around activity, bed positioning, respiratory measures, an anti embolism device, and so on.
As vital signs are frequently documented, they provide a useful visualization of the data in the *chartevents* table. However, to find out which `itemid` values correspond to routinely measured vital signs, we must look for them in the *d_items* table.

In [ ]:
idx = di['category'] == 'Routine Vital Signs'
di.loc[idx]

We can see that there are quite a few columns in the *d_items* table. The `label` column provides a human interpretable description of the `itemid`, while `abbreviation` shows an abbreviated form. The `linksto` column indicates which table contains data associated with the `itemid` (each `itemid` is only ever present in one of the events tables). The remaining columns provide information about the concept: a high level category (`category`), the unit of measure (`unitname`), and the type of measurement (`param_type`, can be 'Text' for drop down items, 'Numeric' for numeric data, 'Date' for timestamp data, or 'Checkbox' for 0/1 data). Normal reference ranges are also provided in the `lownormalvalue` and `highnormalvalue` columns.

Scrolling through this list, we can identify five `itemid` for vital signs to plot:

* 220210 - Respiratory Rate
* 220045 - Heart Rate
* 220179 - Non Invasive Blood Pressure - systolic
* 220180 - Non Invasive Blood Pressure - diastolic
* 220181 - Non Invasive Blood Pressure - mean
* 223762 - Temperature Celsius

We can now plot the vital signs for our patient by selecting only the rows corresponding to each one of these `itemid` values in turn.

N.B. We do not include 220052 (Arterial Blood Pressure mean) as the patient never has it measured (i.e. they never had an indwelling arterial catheter), though it would certainly be a useful measure for other patients.

In [ ]:
plt.figure(figsize=[16, 8])

chart_labels = {
  220045: 'Heart Rate',
  220210: 'Respiratory Rate',
  220179: 'NIBP (sys)',
  220180: 'NIBP (dias)',
  220181: 'NIBP (mean)',
  223761: 'Temperature (deg F)'
}
for itemid, label in chart_labels.items():
  # charted data from the ICU
  idx = ce['itemid'] == itemid
  dff = ce.loc[idx].sort_values('charttime')
  p = plt.plot(dff['charttime'], dff['valuenum'], 'o--', label=label)

plt.legend(loc='upper right')
plt.xticks(rotation=45)

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# plt.xlim(['2110-09-06 18:00:00', '2110-09-08 18:00:00'])
plt.show()

Interestingly we can see a sudden increase in the patient's heart rate just before midnight on the 28th. There are a few possible causes, and we could plot associated data to try to figure it out! An obvious first check is the patient's heart rhythm.

In [ ]:
plt.figure(figsize=[16, 8])

chart_labels = {
  220045: 'Heart Rate',
  220210: 'Respiratory Rate',
  220179: 'NIBP (sys)',
  220180: 'NIBP (dias)',
  220181: 'NIBP (mean)'
}
for itemid, label in chart_labels.items():
  # charted data from the ICU
  idx = ce['itemid'] == itemid
  dff = ce.loc[idx].sort_values('charttime')
  p = plt.plot(dff['charttime'], dff['valuenum'], 'o--', label=label)

markers = ['o', 's', 'd', '^']
rhythms = ce.loc[ce['itemid'] == 220048, 'value'].unique()
for i, rhythm in enumerate(rhythms):
  idx = (ce['itemid'] == 220048) & (ce['value'] == rhythm)
  plt.plot(ce.loc[idx, 'charttime'], np.ones(idx.sum())*80 + i*5, markers[i], label=rhythm)

plt.legend(loc='upper left')
plt.xticks(rotation=45)

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# plt.xlim(['2110-09-06 18:00:00', '2110-09-08 18:00:00'])
plt.show()

This makes it quite clear - the patient has atrial fibrillation charted at 22:00:00 which is around the time the heart rate increases. You may have also noted that paroxysmal atrial fibrillation was one of the diagnoses billed for in the *diagnoses_icd* table earlier.

### Other events tables

With our new understanding of *chartevents*, it's fairly easy to understand the other events tables.

* *datetimeevents* - This table is similar to *chartevents*, but the `value` in this case is a date shifted date which is consistent with the patient's deidentified timeline
* *outputevents* - Contains outputs from the patient: urine output volume, fluid output from drains, stool, and so on.
* *inputevents* - Contains inputs given to the patient either orally or via another route, with intravenous being common. Notably this table does not have `charttime`: each row has a `starttime` and an `endtime`, and inputs are durations rather than instantaneous events.
* *procedureevents* - Similar to *inputevents*, but for processes which are not patient inputs (mechanical ventilation, dialysis, and so on).

Let's look at *inputevents* since it contains useful information about patient administrations and has a slightly different form than *chartevents*.

In [ ]:
# Load admissions
query = f"""
SELECT *
FROM `physionet-data.{dataset}_icu.datetimeevents`
ORDER BY charttime
LIMIT 5
"""
inp = run_query(query)
print(inp.shape)
inp.head()

In [ ]:
# Load admissions
query = f"""
SELECT *
FROM `physionet-data.{dataset}_icu.inputevents`
WHERE stay_id = {stay_id}
ORDER BY starttime, itemid
"""
inp = run_query(query)
print(inp.shape)
inp.head()

As mentioned before, we see we have a `starttime` and an `endtime` for each patient. Note also the large number of extra columns describing the input given including the type, the patient's weight (many medications are dosed by weight), the original amount in the container which held the medication (e.g. the volume of the IV bag), and so on.

Let's look for medications started after 20:00:00 on 2173-03-31.

In [ ]:
df = inp.merge(di[['itemid', 'label']], on='itemid', how='inner')
idx = df['starttime'] >= '2173-03-31 20:00:00'
cols = ['stay_id', 'starttime', 'endtime', 'itemid', 'label', 'amount', 'amountuom', 'rate', 'rateuom']
df.loc[idx, cols].sort_values(['starttime', 'label'])

Note the administration of magnesium - a common treatment for atrial fibrillation. We can plot the administration of the treatment over time combined with the vital signs from earlier to visualize how the timing fits.

In [ ]:
plt.figure(figsize=[16, 8])

chart_labels = {
  220045: 'Heart Rate',
  220210: 'Respiratory Rate'
}
for itemid, label in chart_labels.items():
  # charted data from the ICU
  idx = ce['itemid'] == itemid
  dff = ce.loc[idx].sort_values('charttime')
  p = plt.plot(dff['charttime'], dff['valuenum'], 'o--', label=label)


inp_labels = {
  227523: 'Magnesium'
}
for itemid, label in inp_labels.items():
  # charted data from the ICU
  idx = inp['itemid'] == itemid
  dff = inp.loc[idx].sort_values('starttime')
  for i, row in dff.iterrows():
    amount = pd.to_numeric(row['amount'])
    plt.fill_between([row['starttime'], row['endtime']], [0, 0], [amount, amount], color='#a8ddb5', alpha=0.5)

plt.legend(loc='upper right')
plt.xticks(rotation=45)

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

# plt.xlim(['2110-09-06 18:00:00', '2110-09-08 18:00:00'])
plt.show()


From this, it's hard to conclude whether this is just magnesium supplementation or a treatment for AF - but it is certainly interesting!

You may have also noted that we now have a few different sources of medication information for a patient while they are in the ICU. If we go back to our plots regarding heparin, we can compare the in ICU documentation (*inputevents*) with the hospital level documentation (*emar* and *prescriptions*).

Note that the following code is quite complex! At a high level, we are:

* looking for `itemid` associated with heparin administration in *inputevents* (usually there will only be one `itemid` for a drug)
* separating bolus administrations of this drug (rate is NaN, while amount is non-zero) from the continuous administrations, primarily for visualization purposes
  * if you want the total amount of drug received, you could easily sum across the amount column
* looking for `medication` associated with heparin administration in *emar*
  * again, we separate out bolus administrations from continuous inputs.
  * it's not obvious from the code, but the *emar* table has a number of columns which further describe the administration (e.g. the unit of measure for these are "units" or "units/hour")
* finally, we search for `drug` names for heparin in the *prescriptions* table
  * many of the prescriptions are ranges (1300-2600 units/hour), so the plot is a shaded box with the lower and upper prescribed dose
  * some prescriptions are for large IV mixes of 25,000 units - plot these at 2500 to help the visualization

In [ ]:
plt.figure(figsize=[16, 12])

# ICU documentation (inputevents) of heparin administration
inp_labels = {
  225152: 'Heparin (inputevents)'
}

for itemid, label in inp_labels.items():
  idx = inp['itemid'] == itemid
  dff = inp.loc[idx].sort_values('starttime')
  # first, plot bolus heparin
  idx = dff['rate'].isnull()
  plt.plot(dff.loc[idx, 'starttime'], dff.loc[idx, 'amount'], 'd', markersize=12, label='Heparin bolus (inputevents)')

  for i, row in dff.iterrows():
    rate = pd.to_numeric(row['rate'])
    plt.plot([row['starttime'], row['endtime']], [rate, rate], 'o-', linewidth=4, color='#fc8d62', label=label)
    # disable legend plotting in future rows
    label = '__no_legend__'
    # plt.fill_between([row['starttime'], row['endtime']], [0, 0], [rate, rate], color='#fc8d62', alpha=0.8)

# eMAR documentation of heparin
med = 'Heparin'
idx_med = (df_emar['medication'] == med)
# first, plot bolus separately
idx = idx_med & (df_emar['administration_type'] == 'Heparin IV Bolus')
plt.plot(df_emar.loc[idx, 'charttime'], df_emar.loc[idx, 'dose_due'], 's', color='#80a0cb', markersize=16, alpha=0.5, label='Heparin bolus (eMAR)')

idx = idx_med & (df_emar['administration_type'] != 'Heparin IV Bolus')
plt.plot(df_emar.loc[idx, 'charttime'], df_emar.loc[idx, 'dose_due'], 'o', color='#1f78b4', markersize=12, alpha=0.9, label='Heparin (eMAR)')

# prescriptions documentation of heparin
meds = ['Heparin', 'Heparin Sodium']
colors = {
    'Heparin': '#1b9e77',
    'Heparin Sodium': '#1b9e77',
}
idx = pr['drug'].isin(meds)
df = pr.loc[idx].copy()

for med in meds:
  label = med + ' (prescriptions)'
  color = colors[med]
  idx = df['drug'] == med
  n = 0
  for i, row in df.loc[idx].iterrows():
    if n > 0:
      # do not add to the legend if we have already plotted at least once
      med = '__no_legend__'

    xi = [row['starttime'], row['stoptime']]
    dose = row['dose_val_rx']
    if '-' in dose:
      # plot upper/lower ranges
      dose_lower, dose_upper = dose.split('-')
      dose_lower = int(dose_lower)
      dose_upper = int(dose_upper)
      plt.fill_between(xi, [dose_lower, dose_lower], [dose_upper, dose_upper], color=color, alpha=0.5, label=label)
    else:
      dose = pd.to_numeric(dose.replace(',', ''))
      dose = dose / 10
      plt.plot(xi, [dose, dose], '*-', color=color, label=label)
    label = '__no_legend__'

    n += 1


# shade in their ICU stay
upper = 3000
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [upper, upper], color='#ababab', alpha=0.2)

plt.legend(loc='upper right')
plt.xticks(rotation=45)
plt.ylabel('Heparin administration (units/hour)')

# concise dates on the x-axis
ax = plt.gca()
locator = mdates.AutoDateLocator(minticks=3, maxticks=7)
formatter = mdates.ConciseDateFormatter(locator)
ax.xaxis.set_major_locator(locator)
ax.xaxis.set_major_formatter(formatter)

plt.show()

**Note in the above the Heparin Sodium (prescriptions) are 25,000 units, downscaled by a factor of 10 for clarity.**

This plot highlights how useful administration data is. Prescription information describes the intention to treat, while administration describes the actual treatment given. This is very common in MIMIC-IV, particularly for medications which are titrated based on physiology. Note also the relatively good concordance between the manually documented ICU data in *inputevents* and the barcode scanned infusion administration information from *emar*.

## That's MIMIC-IV!

With the final comparison of the ICU and hospital level medication administration data, you'll now have a pretty good idea of the type information in MIMIC-IV.

But we're not done yet! One of the most interesting aspects of MIMIC-IV is that it uses the same patient identifiers as the recently published MIMIC-CXR database. Of course this is intentional to allow linking the to datasets!

## Chest x-rays for this patient

The `mimic_cxr` schema does not contain any of the data for MIMIC-CXR. Instead, it provides a useful mapping between the DICOM filenames and the patient anonymous identifiers.

The tables in `mimic_cxr` are:

* *study_list* - `subject_id`, `study_id`, `path`
  * Contains the relative path to individual radiology reports. Each radiology report is represented by a unique `study_id`. `study_id` is unique in this table (each row has a distinct `study_id`).
* *record_list* - `subject_id`, `study_id`, `dicom_id`, `path`
  * Contains the relative path to individual DICOMs. Each DICOM (represented by `dicom_id`) stores a chest x-ray related to a single radiographic study (represented by `study_id`). `dicom_id` is unique in this table (each row has a distinct `dicom_id`).
* *dicom_metadata_string* - `dicom`, ... (many columns corresponding to DICOM meta-data fields).

In order to access the `mimic_cxr` schema, you must request it via [the project page on PhysioNet](https://physionet.org/content/mimic-cxr/).

In [ ]:
query = f"""
select
  subject_id, study_id, path
from `physionet-data.mimic_cxr.study_list`
where subject_id = {subject_id}
ORDER BY study_id
"""
cxr_studies = run_query(query)
display(cxr_studies)

From the above, we can see that this subject had 4 distinct radiology studies which corresponded to chest x-rays. We do not know the order of these studies, or when they were performed, as the `study_id` is a random integer.

We can look for the unique DICOMs associated with this patient as well.

In [ ]:
query = f"""
select
  subject_id, study_id, dicom_id, path
from `physionet-data.mimic_cxr.record_list`
where subject_id = {subject_id}
ORDER BY study_id, dicom_id
"""
cxr_records = run_query(query)
display(cxr_records)

We now have six rows. This occurs because we often have more than one image associated with a single radiology study. This is common in chest x-rays, where a frontal view and a side view are standard. The frontal view can be "PA", standing for posterior-anterior (the image is a projection of the patient facing *away* from the scanner), or "AP" for anterior-posterior (the image is a projection of the patient facing *toward* the scanner). This has a bigger impact on the image than you might think initially, due to the physics of x-ray scanning

![AP vs PA chest x-ray.](https://github.com/MIT-LCP/mimic-cxr/raw/master/website/static/img/ap_vs_pa_opt.jpg)

As you can see, an AP image would result in a larger shadow of the heart on the x-ray scanner when compared to a PA scan. As a result, the orientation of the patient is important in correctly determining the size of the heart (and this is just the most obvious difference!).

Happily, the DICOM header provides information about the patient's orientation, and we have loaded most of the information in the DICOM header into a table on BigQuery. There are *many* columns in this table, but we can subselect the ones relevant for orientation.


In [ ]:
query = f"""
SELECT
  dicom, StudyID, PatientID, StudyDate, StudyTime, BodyPartExamined, ViewPosition, `Rows`, Columns, BurnedInAnnotation
from `physionet-data.mimic_cxr.dicom_metadata_string`
where PatientID = '{subject_id}'
ORDER BY 4, 5
"""
cxr_metadata = run_query(query)
display(cxr_metadata)

Note the slight changes in schema, as this table uses DICOM standard field names. We can see that these are all chest x-rays (`BodyPartExamined`), and one is explicitly noted as a portable chest x-ray ("PORT CHEST"). The view position is AP for most images, though one is a lateral x-ray (all side profiles for chest x-rays have the left side of the patient next to the image receptor, so "LEFT LATERAL" and "LATERAL" are synonymous).

Note that we have x-rays from before their ICU admission (2176-11-28), as the patient visited the emergency room (which we saw in the *transfers* table earlier). We'll focus on their x-rays from the same date as their ICU admission.

We'll now subselect to the studies performed on 2176-11-28.

In [ ]:
idx = cxr_metadata['StudyDate'] == '21761128'
cxr_metadata = cxr_metadata.loc[idx]

cxr_records = cxr_records.merge(cxr_metadata, left_on='dicom_id', right_on='dicom', how='inner')
display(cxr_records)

We will also subselect the studies in the `cxr_studies` dataframe.

In [ ]:
cxr_studies = cxr_studies.loc[cxr_studies['study_id'].isin(cxr_records['study_id'])]
display(cxr_studies)

As we can see above, there was a single x-ray performed. We can use the relative pathname provided in the `path` column in combination with the Google Cloud Storage location of the data to load individual reports/images. This requires your PhysioNet account to be linked to your Google account, and requires you to specifically request access to MIMIC-CXR on Google Cloud via the [MIMIC-CXR PhysioNet page](https://physionet.org/content/mimic-cxr/).

In [ ]:
import os

cxr_location = 'gs://mimic-cxr-2.0.0.physionet.org/'

# print out the studies
for i, cxr_study in cxr_studies.iterrows():
  # load in the text
  path = cxr_study['path']
  study_id = cxr_study["study_id"]

  # the filename is the study_id with .txt extension
  # e.g. s########.txt
  fn = f's{study_id}.txt'

  # download the report if it hasn't been downloaded yet
  if not os.path.exists(fn):
    !gsutil cp {cxr_location}{path} ./

  with open(fn, 'r') as fp:
    txt = ''.join(fp.readlines())

  print(f'\n\n{"="*35} {study_id} {"="*35}\n')
  print(txt)

This gives us a fairly good idea of why the x-ray was performed. The patient was hypercapnic and they were assessing for some kind of abnormality which might explain the symptom. We can use a similar method to look at the chest x-ray itself!

In [ ]:
!pip install pydicom
import pydicom

In [ ]:
# first DICOM study
for i, row in cxr_records.iterrows():
  dicom_id = row['dicom_id']
  dicom_path = row['path']
  fn = f'{dicom_id}.dcm'

  # download the image if it hasn't been downloaded yet
  if not os.path.exists(fn):
    # the DICOM is located at: {cxr_location}{path}
    # this is of the form:
    # gs://<bucket_name>/p10/p1######/s5#######/<dicom_id>.dcm
    # where p1###### is the patient identifier
    # and s5####### is the study identifier
    path = row['path']
    !gsutil cp {cxr_location}{path} ./

  with open(fn, 'rb') as fp:
    plan = pydicom.dcmread(fp)

  # print the DICOM plan - this is *long*!
  print(plan)

  # plot the image
  plt.figure(figsize=[16,10])
  plt.imshow(plan.pixel_array)
  plt.grid(False)
  plt.axis('off')
  # set colormap to bone
  plt.bone()
  plt.show()

For convenience, the MIMIC-CXR-JPG dataset contains all images in the MIMIC-CXR dataset preprocessed and converted to JPG. We can load these images as well. As JPG is a compressed format, the files are much smaller, though at a loss of resolution.

In [ ]:
from PIL import Image
jpg_location = 'gs://mimic-cxr-jpg-2.0.0.physionet.org/'

# first DICOM study
for i, row in cxr_records.iterrows():
  dicom_id = row['dicom_id']
  fn = f'{dicom_id}.jpg'

  # download the image if it hasn't been downloaded yet
  if not os.path.exists(fn):
    path = row['path']
    # the DICOM is located at: {cxr_location}{path}
    !gsutil cp {jpg_location}{path.replace('.dcm', '.jpg')} ./


  # original image
  print(fn)
  image = Image.open(fn)
  fig = plt.figure(figsize=(16,12))
  plt.imshow(image)
  plt.axis('off')
  plt.bone()
  plt.show()

Note that despite using the same colormap, the JPG and DICOM images appear very slightly different. This occurs because histogram equalization was performed when creating the JPG data. You can read more about the JPG subset on [the PhysioNet project page for MIMIC-CXR-JPG](https://physionet.org/content/mimic-cxr-jpg/2.0.0/).

### Combining MIMIC-CXR with MIMIC-IV

One important aspect of MIMIC-CXR and MIMIC-IV are that the dates and times are aligned, meaning the time of an x-ray in the DICOM header for MIMIC-CXR matches the time at which it was performed in MIMIC-IV. Recall that the *poe* table mentioned a few chest x-ray orders - let's look for those rows.

In [ ]:
poe.loc[poe['order_subtype'] == 'General Xray']

This says there was an x-ray order at 2176-11-28 14:40:28. We can then look at the metadata in the DICOM header (currently stored in cxr_records) to see if that matches expectation.

In [ ]:
cxr_records[['dicom_id', 'subject_id', 'StudyDate', 'StudyTime']]

This is telling us that there was an order for an x-ray around 2:40pm, and a chest x-ray study was performed at 3:09pm. Great!

We can visualize where this occurred in the patient's hospital course.

In [ ]:
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox


plt.figure(figsize=[16, 8])
ax = plt.gca()

# labs
lab_labels = [
  'pO2',
  'pCO2',
  'Calculated Total CO2',
]

for label in lab_labels:
  idx = (lab['label'] == label) & (lab['fluid'] == 'Blood')
  plt.plot(lab.loc[idx, 'charttime'], lab.loc[idx, 'valuenum'], 'o--',  label=f'{label}')

# add specimen type
for i, row in lab.loc[lab['label'] == 'Specimen Type'].iterrows():
  plt.text(row['charttime'], 90, row['value'], rotation=60, ha='center')


# add in the x-rays
for i, row in cxr_records.iterrows():
  dicom_id = row['dicom_id']
  fn = f'{dicom_id}.jpg'

  # get the date/time
  studytime = pd.to_datetime('T'.join([row['StudyDate'], row['StudyTime']]))

  # load in the JPG
  image = Image.open(fn)

  # place it on the axis in an annotation box
  imagebox = OffsetImage(image, zoom=0.06)
  ab = AnnotationBbox(imagebox, (studytime, 25 + i*50), pad=0)
  ax.add_artist(ab)

  if i == 0:
    plt.plot(studytime, 0, 's', markersize=12, label='Chest x-ray measured')

# shade in their ICU stay
for i, row in tr.iterrows():
  if row['careunit'] is None:
    continue
  if 'ICU' in row['careunit']:
    plt.fill_between([row['intime'], row['outtime']], [0, 0], [100, 100], color='#ababab', alpha=0.3)

plt.legend(fontsize=16, loc='lower right')
plt.xticks(rotation=45)
plt.show()

So it appears that this was an x-ray done while the patient was in hospital, but *after* their ICU stay, probably just to check everything was going OK.